# Evaluating annotations of understatements



In [1]:
# Install necessary libraries
import sys
libraries = ["polars", "openai", "instructor", "pydantic", "statsmodels", "os"]

for library in libraries:
    try:
        __import__(library)
        print(f"{library} is already installed.")
    except ImportError:
        print(f"{library} not found, installing...")
        !{sys.executable} -m pip install {library}

polars not found, installing...
zsh:1: no such file or directory: /Users/b/Documents/Digital
openai not found, installing...
zsh:1: no such file or directory: /Users/b/Documents/Digital
instructor not found, installing...
zsh:1: no such file or directory: /Users/b/Documents/Digital
pydantic not found, installing...
zsh:1: no such file or directory: /Users/b/Documents/Digital
statsmodels not found, installing...
zsh:1: no such file or directory: /Users/b/Documents/Digital
os is already installed.


In [8]:
from pydantic import BaseModel
from typing_extensions import Literal
from openai import OpenAI
import polars as pl
import instructor
import statsmodels
import os
import dotenv

In [9]:
# Set your key and a model from https://platform.openai.com/docs/models/continuous-model-upgrades
chosen_model = "gpt-4-turbo"
dotenv.load_dotenv('.env')
key = os.environ.get('OPENAI_API_KEY')
client = OpenAI(api_key=key)

## Part 1: Annotating understatements with LLMs

### Step 1: Import and preprocess data

The sentences for the annotation task are in a Google Sheet and exported as TSV. 

To annotate the sentences:
1. Open the TSV files
2. Extract the sentences from the TSV file

In [10]:
# Open the TSV file.
with open('datasets/annotation-task.tsv', 'r') as file:
    text = file.read()

# Extract the sentences from the TSV file.
rows = text.split('\n')
data = [row.split('\t') for row in rows]
unlabeled_dataset = data[1:]

### Step 2: Test the understatement annotation task

Let's test a simple prompt for the understatement annotation task.

In the prompt, we present a phrase in context and ask for four labels:

1. Whether the sentence is an understatement (Yes/No)
2. *It's confidence in the label (1-3)
3. If it thinks the phrase is an understatement, whether its litoses or meiosis (Litotes/Meiosis)
4. If it thinks the phrase is an understatement, the pragmatic function of the understatement (Mocking/Humorous/Tempering)

The phrase in context is surrounded with a pair of square brackets.


In [15]:
first_example = unlabeled_dataset[60][1]
prompt = f"""Read the following text and answer four questions: 

---START OF TEXT---
{first_example}
---END OF TEXT---

Question 1: Is the phrase in square brackets an understatement?
Question 2: On a scale from 1 to 3, how confident are you in your answer to question 1?
Question 3: If you think the phrase is an understatement, is it type litoses or meiosis?
Question 4: If you think the phrase is an understatement, is the pragmatic function of the understatement mocking, humorous, or tempering?"""

print(prompt)

Read the following text and answer four questions: 

---START OF TEXT---
“Do you know, my friends,” said Michel Ardan, “that if one of us had succumbed to the shock consequent on departure, we should have had a great deal of trouble to bury him? What am I saying? to etherize him, as here ether takes the place of earth. You see the accusing body would have followed us into space like a remorse.”  “[That would have been sad],” said Nicholl.
---END OF TEXT---

Question 1: Is the phrase in square brackets an understatement?
Question 2: On a scale from 1 to 3, how confident are you in your answer to question 1?
Question 3: If you think the phrase is an understatement, is it type litoses or meiosis?
Question 4: If you think the phrase is an understatement, is the pragmatic function of the understatement mocking, humorous, or tempering?


Next, we create a helper function to pass the prompt to the LLM:

In [16]:
# Here's how we interact with the model
def annotate(chosen_prompt):
    resp = client.chat.completions.create(
        model=chosen_model, 
        messages=[
            {"role": "user", 
            "content": chosen_prompt},
        ],
        temperature=1,
        )
    annotation = resp.choices[0].message.content
    return annotation

Let's test the prompt with our first sentence:

In [17]:
print(annotate(prompt))

Answer 1: Yes, the phrase in square brackets, "That would have been sad," is an understatement.

Answer 2: I am 3 (very confident) in my answer to question 1. The expression "That would have been sad" minimizes the emotional impact and the unusual, eerie scenario described, making it a clear understatement.

Answer 3: The understatement is an example of meiosis. Meiosis is a type of understatement that deliberately understates or de-emphasizes the importance of something. In this case, describing the potential presence of an "accusing body" haunting them in space as merely "sad" significantly downplays the scenario.

Answer 4: The pragmatic function of the understatement in this context is likely humorous. The use of such a casual remark in the context of a bizarre and potentially horrifying situation can add a touch of irony and levity, suggesting that the speaker is using humor to deal with the concept of death and its consequences in a detached, unusual environment like space.


Done. While the model has replied to all four questions, we have two issues:

1. The model only annotated one sentence. We need to annotate **all** the sentences.
2. The model's answers are unstructured. We need to extract the answers and save them in a structured format.

Let's solve these issues one by one.

### Step 3: Annotate many texts (understatements)

If we iterate over all the sentences, we can annotate them one by one and save the annotations in a list called `annotations_list`.

In [18]:
annotations_list = []

for i, row in enumerate(unlabeled_dataset, 1):
    
    prompt = f"""Read the following text and answer four questions: 

---START OF TEXT---
{row[1]}
---END OF TEXT---

Question 1: Is the phrase in square brackets an understatement?
Question 2: On a scale from 1 to 3, how confident are you in your answer to question 1?
Question 3: If you think the phrase in square brackets is an understatement, is it type litoses or meiosis?
Question 4: If you think the phrase in square brackets is an understatement, is the pragmatic function of the understatement mocking, humorous, or tempering?"""
    
    print(f" --- ANNOTATING TEXT {i} ---")
    annotation = annotate(prompt)
    print(annotation)
    print("\n")

    # store the annotations in the dictionary
    annotations_list.append(annotation)

 --- ANNOTATING TEXT 1 ---
Answer 1: Yes, the phrase in square brackets ("[I might have dressed up a little for the evening]") is an understatement.

Answer 2: 3 - I am very confident in the answer to question 1. The description of the attire as immensely elaborate and eye-catching directly contradicts the minimalistic language used in the phrase, thus qualifying it as an understatement.

Answer 3: The understatement is a type of meiosis. Meiosis involves understating or diminishing the importance of something, which in this case, is the elaborate nature of Mrs. Allen's attire, described as just "a little."

Answer 4: The pragmatic function of the understatement seems to be humorous. By dramatically downplaying the elaborateness of her attire in such a grand setting, Mrs. Allen likely aims to amuse her audience and add a touch of light-heartedness to her grand entrance.


 --- ANNOTATING TEXT 2 ---
Answer 1: Yes, the phrase in square brackets, "not unremarkable," is an understatement.


KeyboardInterrupt: 

### Step 4: Annotating understatements with structured output

Now that we can run the prompt for all samples in the dataset, we need to extract the answers and save them in a structured format.

To do this, we first need to amend the prompt to elicit structured responses. We do this by commanding the model to respond with a JSON object only:

In [20]:
prompt = f"""Read the following text and answer four questions: 

---START OF TEXT---
{first_example}
---END OF TEXT---

Question 1: Is the phrase in square brackets an understatement?
Question 2: On a scale from 1 to 3, how confident are you in your answer to question 1?
Question 3: If you think the phrase in square brackets is an understatement, is it type litoses or meiosis?
Question 4: If you think the phrase in square brackets is an understatement, is the pragmatic function of the understatement mocking, humorous, or tempering?

Respond with a JSON object only. Do not return any other information."""

To ensure the LLMs responses adhere to a structured format, we need to define a data model.

All responses are then matched to the data model, ensuring uniform structure. 

We use `pydantic` to define a data model, named `Understatement` and with four attributes:

1. `is_understatement`: Whether the sentence is an understatement (Yes/No)
2. `confidence`: The model's confidence in the label (1-3)
3. `understatement_type`: If the phrase is an understatement, whether it's litotes or meiosis (Litotes/Meiosis)
4. `pragmatic_function`: If the phrase is an understatement, the pragmatic function of the understatement (Mocking/Humorous/Tempering)



In [21]:
class Understatement(BaseModel):
    """
    An understatement with its annotations.
    """
    is_understatement: bool
    confidence: Literal[1, 2, 3]
    understatement_type: Literal["litotes", "meiosis", None]
    pragmatic_function: Literal["mocking", "humorous", "tempering", None]

Next, we initialize a client for `instructor` to extract the answers from the LLM's output and validate them against the `Understatement` model:

In [22]:
client = instructor.patch(OpenAI(api_key=key), mode=instructor.Mode.MD_JSON)

Let's update our helper function to use the `instructor` client, and pass the `Understatement` model to validate the responses:

In [23]:
def annotate_structured(chosen_prompt):
    resp = client.chat.completions.create(
        model=chosen_model, 
        messages=[
            {"role": "user", 
            "content": chosen_prompt},
        ],
        temperature=1,
        response_model=Understatement,
        )
    return resp

Finally, let's use the updated helper function to get a structured `annotation`:

In [24]:
annotation = annotate_structured(prompt)
print(annotation)

is_understatement=True confidence=2 understatement_type='meiosis' pragmatic_function='humorous'


The result is an `Understatement` object, with four attributes, corresponding to the four labels assigned per sample. 

These labels are chosen from closed sets of labels, of boolean (`is_understatement`), categorical (`understatement_type` and `pragmatic_function`), and ordinal (`confidence`) type.

### Step 5: Annotate many understatements with structured responses

Let's combine the steps above to annotate all the sentences in the dataset and save the annotations in a structured format.

In [48]:
annotated_data = []

for row in unlabeled_dataset:
    element = {}
    
    prompt = f"""Read the following text and answer four questions: 

---START OF TEXT---
{row[1]}
---END OF TEXT---

Question 1: Is the phrase in square brackets an understatement?
Question 2: On a scale from 1 to 3, how confident are you in your answer to question 1?
Question 3: If you think the phrase in square brackets is an understatement, is it type litoses or meiosis?
Question 4: If you think the phrase in square brackets is an understatement, is the pragmatic function of the understatement mocking, humorous, or tempering?

Respond with a JSON object only."""
    
    try:
        annotation = annotate_structured(prompt)
    except Exception as e:
        # If the model fails to return a valid JSON object try again
        print(f"Error: {e}")
        annotation = annotate_structured(prompt)

    print(f" --- ANNOTATING TEXT {row[-1]} ---")
    print(annotation)

    element["phrase"] = row[0]
    element["full_text"] = row[1]
    element["is_understatement"] = annotation.is_understatement
    element["confidence"] = annotation.confidence
    element["understatement_type"] = annotation.understatement_type
    element["pragmatic_function"] = annotation.pragmatic_function
    annotated_data.append(element)

 --- ANNOTATING TEXT  ---
is_understatement=True confidence=3 understatement_type='meiosis' pragmatic_function='humorous'
 --- ANNOTATING TEXT  ---
is_understatement=True confidence=3 understatement_type='litotes' pragmatic_function='mocking'
 --- ANNOTATING TEXT  ---
is_understatement=True confidence=3 understatement_type='meiosis' pragmatic_function='tempering'
 --- ANNOTATING TEXT  ---
is_understatement=True confidence=3 understatement_type='meiosis' pragmatic_function='tempering'


KeyboardInterrupt: 

Before we move on, let's save the annotations in a JSON file:

In [34]:
# Store annotated_data as JSON

import json


In [ ]:

with open('datasets/annotated-data-llm_baseline.json', 'w') as file:
    json.dump(annotated_data, file, indent=4)

Since we may want to rerun this notebook without having to reannotate everything through OpenAI again, we can simply load what we have saved:

In [35]:
with open('datasets/annotated-data-llm_baseline.json', 'r') as file:
    annotated_data = json.load(file)

Done. Let's inspect the results:

In [36]:
annotations_baseline_llm = annotated_data

for i, sentence in enumerate(annotations_baseline_llm, 1):
    print(f" --- ANNOTATED TEXT {i} ---")
    print(f'phrase: "{sentence["phrase"]}"')
    print(f"is_understatement: {sentence['is_understatement']}")
    print(f"confidence: {sentence['confidence']}")
    print(f"understatement_type: {sentence['understatement_type']}")
    print(f"pragmatic_function: {sentence['pragmatic_function']}")
    print("\n")

 --- ANNOTATED TEXT 1 ---
phrase: "I might have dressed up a little for the evening"
is_understatement: True
confidence: 3
understatement_type: meiosis
pragmatic_function: humorous


 --- ANNOTATED TEXT 2 ---
phrase: "not unremarkable"
is_understatement: True
confidence: 3
understatement_type: litotes
pragmatic_function: mocking


 --- ANNOTATED TEXT 3 ---
phrase: "took a tiny interest"
is_understatement: True
confidence: 3
understatement_type: meiosis
pragmatic_function: tempering


 --- ANNOTATED TEXT 4 ---
phrase: "Sometimes he's not quite the same, but we've done pretty good work."
is_understatement: True
confidence: 3
understatement_type: meiosis
pragmatic_function: tempering


 --- ANNOTATED TEXT 5 ---
phrase: "And probably twice as cursed"
is_understatement: True
confidence: 3
understatement_type: meiosis
pragmatic_function: humorous


 --- ANNOTATED TEXT 6 ---
phrase: "the stars themselves are hardly dead"
is_understatement: True
confidence: 3
understatement_type: meiosis
pragm

Now that we have successfully annotated all the sentences with an LLM, we can load the human annotations and compute Inter-Annotator Agreement metrics.

## Part 2: Load data from human annotators

Let's load the annotations from human annotators and normalize the labels to match the LLM annotations.

In [44]:
def extract_annotation_dict(row):
    """
    Extracts the data from a row of the TSV file.
    
    Args:
    - row: a list with the data of a row of the TSV file
    """

    sample = {}
    sample['phrase'] = row[0]
    sample['full_text'] = row[1]
    sample['is_understatement'] = row[2]
    sample['confidence'] = row[3]
    sample['pragmatic_function'] = row[4]
    sample['understatement_type'] = row[5]
    return sample

def normalize_annotation(sample):
    """
    Normalizes an annotation to match `Understatement` model.

    Args:
    - sample: a dictionary with the sample data
    """

    # Normalize `is_understatement`.
    if sample['is_understatement'] == "Yes":
        sample['is_understatement'] = True
    if sample['is_understatement'] == "No":
        sample['is_understatement'] = False
    if sample['is_understatement'] == "":
        sample['is_understatement'] = None

    # Normalize `confidence`.
    if sample['confidence'] == '1 - Not confident':
        sample['confidence'] = 1
    if sample['confidence'] == '2 - Somewhat confident':
        sample['confidence'] = 2
    if sample['confidence'] == '3 - Confident':
        sample['confidence'] = 3
    if sample['confidence'] == "":
        sample['confidence'] = None

    # Normalize `pragmatic_function`.
    if sample['pragmatic_function'] == "Tempering":
        sample['pragmatic_function'] = "tempering"
    if sample['pragmatic_function'] == "Humoristic":
        sample['pragmatic_function'] = "humorous"
    if sample['pragmatic_function'] == "Derisiv/mocking":
        sample['pragmatic_function'] = "mocking"
    if sample['pragmatic_function'] == "":
        sample['pragmatic_function'] = None

    # Normalize `understatement_type`.
    if sample['understatement_type'] == "Litotes (a negation of the opposite)":
        sample['understatement_type'] = "litotes"
    if sample['understatement_type'] == "Meiosis (a weaker expression)":
        sample['understatement_type'] = "meiosis"
    if sample['understatement_type'] == "":
        sample['understatement_type'] = None

    return sample

def get_annotations(path):
    """
    Extracts normalized annotations from a TSV file.

    Args:
    - path: the path to the TSV file
    """
    annotations = []

    # Open the TSV file, read the text, and split it into rows.
    with open(path, 'r') as file:
        text = file.read()
        rows = text.split('\n')
        data = [row.split('\t') for row in rows[1:]]
        
    # Extract the annotations from the rows, and normalize them.
    for row in data:
        sample = extract_annotation_dict(row)
        sample = normalize_annotation(sample)
        annotations.append(sample)

    return annotations

# Get annotations from four different annotators.
annotations_szabi = get_annotations('datasets/annotation-task-szabi.tsv')
annotations_antanas = get_annotations('datasets/annotation-task-antanas.tsv')
annotations_kris = get_annotations('datasets/annotation-task-kris.tsv')
annotations_bastiaan = get_annotations('datasets/annotation-task-bastiaan.tsv')

## Part 3: Calculate Inter-Annotator Agreement (IAA)

There are multiple metrics to calculate Inter-Annotator Agreement (IAA). 

We will start with the simplest metric, the **Observed Agreement**. Then, we will point out some of its limitations and introduce more advanced metrics.

### Step 1: Calculate Observed Agreement

The simplest metric for IAA is the observed agreement. It is the proportion of times the two annotators agree on the label.

Let's calculate the observed agreement for the `is_understatement` label between two annotators.

In [45]:
def calculate_observed_agreeement(annotations_a, annotations_b, silent=False):
    """
    Calculates the observed agreement between two annotators.
    """
    matches = 0
    for a, b in zip(annotations_a, annotations_b):
        if a['is_understatement'] == b['is_understatement']:
            matches += 1

    if not silent:
        print(f"Number of agreements:\t\t{matches}")
        print(f"Number of annotations:\t\t{len(annotations_szabi)}")
        print(f"Observed agreement (Po):\t{matches / len(annotations_szabi) * 100:.2f}% ({matches}/{len(annotations_szabi)})\n")

    return matches / len(annotations_szabi)

observed_agreement_antanas_szabi = calculate_observed_agreeement(annotations_antanas, annotations_szabi)

Number of agreements:		92
Number of annotations:		120
Observed agreement (Po):	76.67% (92/120)



While the observed agreement is easy to compute, it doesn't account for:

- **Agreement by chance**:  Agreement by chance is the probability that two annotators agree on a label just by chance. It is calculated as the product of the probability that each annotator assigns a label to a text.
- **Difficulty of the task**: If the task is easy, easy tasks will always have higher agreement than difficult tasks.
- **The number of labels**: If the task has many labels, the observed agreement will be lower than if the task has few labels.
- **The number of annotators**: If there are many annotators, the observed agreement will be lower than if there are few annotators.

To demonstrate this, let's calculate the observed agreement betweeen three annotators, instead of two:

In [46]:
matches = 0
for s, a, k, b, llm in zip(annotations_szabi, annotations_antanas, annotations_kris, annotations_bastiaan, annotations_baseline_llm):
    if a['is_understatement'] == s['is_understatement'] == k['is_understatement'] == b['is_understatement'] == llm['is_understatement']:
        matches += 1

print(f"Number of agreements:\t\t{matches}")
print(f"Number of annotations:\t\t{len(annotations_szabi)}")
print(f"Observed agreement (Po):\t{matches / len(annotations_szabi) * 100:.2f}% ({matches}/{len(annotations_szabi)})")

Number of agreements:		79
Number of annotations:		120
Observed agreement (Po):	65.83% (79/120)


While two annotators agreed on 92 out of 120 labels (76.67%), three annotators agreed on only 79 out of 120 labels (65.83%). 

> What does this drop in agreement say about the **quality** of the annotations?

### Step 2: Calculate Cohen's Kappa

Cohen's Kappa is a more advanced metric that accounts for chance agreement (Pe).

To calculate the chance agreement:

1. We need to calculate the probability that each annotator assigns a label to a text.
2. We multiply the probabilities to get the **chance agreement**.

Let's start by calculating the probability that each annotator assigns a label to a text:

In [47]:
true_count_szabi = 0
for s in annotations_szabi:
    if s['is_understatement'] == True:
        true_count_szabi += 1

print("Annotator 1:")
print(f"Phrases labeled as understatement:\t{true_count_szabi}")
print(f"Probability annotator 1 labels as True:\t{true_count_szabi / len(annotations_szabi) * 100:.2f}%\n")

true_count_antanas = 0
for s in annotations_antanas:
    if s['is_understatement'] == True:
        true_count_antanas += 1

print("Annotator 2:")
print(f"Phrases labeled as understatement:\t{true_count_antanas}")
print(f"Probability annotator 2 labels as True:\t{true_count_antanas / len(annotations_antanas) * 100:.2f}%\n")

Annotator 1:
Phrases labeled as understatement:	66
Probability annotator 1 labels as True:	55.00%

Annotator 2:
Phrases labeled as understatement:	63
Probability annotator 2 labels as True:	52.50%



Next, let's define functions to calculate the chance agreement (Pe):

In [29]:
def calculate_chance_agreement(annotations_a, annotations_b):
    """
    Calculates the chance agreement between two annotators.
    """
    
    true_count_a = 0
    true_count_b = 0
    for a, b in zip(annotations_a, annotations_b):
        if a['is_understatement'] == True:
            true_count_a += 1
        if b['is_understatement'] == True:
            true_count_b += 1

    proportion_a = true_count_a / len(annotations_a)
    proportion_b = true_count_b / len(annotations_b)
    Pe = (proportion_a * proportion_b) + ((1 - proportion_a) * (1 - proportion_b))

    return Pe

Finally, let's calculate Cohen's Kappa, using the chance agreement (Pe) and the observed agreement (Po):

In [30]:
def calculate_kappa(annotations_a, annotations_b):
    """
    Calculates the Kappa coefficient for two annotators.

    Args:
    - annotations_a: annotations from the first annotator
    - annotations_b: annotations from the second annotator
    """

    # Calculate the chance agreement (Pe)
    Pe = calculate_chance_agreement(annotations_a, annotations_b)

    # Calculate the observed agreement (Po)
    Po = calculate_observed_agreeement(annotations_a, annotations_b, silent=True)

    # Calculate the Kappa coefficient.
    K = (Po - Pe) / (1 - Pe)
    return K

K_szabi_antanas = calculate_kappa(annotations_szabi, annotations_antanas)
K_szabi_kris = calculate_kappa(annotations_kris, annotations_szabi)
K_antanas_kris = calculate_kappa(annotations_kris, annotations_antanas)

print(f"Kappa coefficient between annotator 1 and annotator 2: ~{K_szabi_antanas:.2f}")
print(f"Kappa coefficient between annotator 1 and annotator 3: ~{K_szabi_kris:.2f}")
print(f"Kappa coefficient between annotator 2 and annotator 3: ~{K_antanas_kris:.2f}")

avg_K = (K_szabi_antanas + K_szabi_kris + K_antanas_kris) / 3

print(f"\nAverage Kappa coefficient: ~{avg_K:.2f}")

Kappa coefficient between annotator 1 and annotator 2: ~0.53
Kappa coefficient between annotator 1 and annotator 3: ~0.61
Kappa coefficient between annotator 2 and annotator 3: ~0.45

Average Kappa coefficient: ~0.53


In [38]:
K_szabi_llm = calculate_kappa(annotations_szabi, annotations_baseline_llm)
K_kris_llm = calculate_kappa(annotations_kris, annotations_baseline_llm)
K_antanas_llm = calculate_kappa(annotations_antanas, annotations_baseline_llm)
K_bastiaan_llm = calculate_kappa(annotations_bastiaan, annotations_baseline_llm)

print(f"Kappa coefficient between annotator 1 and annotator 2: ~{K_szabi_llm:.2f}")
print(f"Kappa coefficient between annotator 1 and annotator 3: ~{K_kris_llm:.2f}")
print(f"Kappa coefficient between annotator 2 and annotator 3: ~{K_antanas_llm:.2f}")
print

Kappa coefficient between annotator 1 and annotator 2: ~0.41
Kappa coefficient between annotator 1 and annotator 3: ~0.30
Kappa coefficient between annotator 2 and annotator 3: ~0.33


The average Kappa score of ~0.53 indicates a moderate agreement.

![Kappa score interpretation](https://www.researchgate.net/profile/Aniekan_Eyoh/publication/268369407/figure/download/tbl3/AS:667837051973633@1536236169043/Interpretation-of-Kappa-statistic.png)


_Source: https://www.researchgate.net/profile/Aniekan_Eyoh/publication/268369407/figure/download/tbl3/AS:667837051973633@1536236169043/Interpretation-of-Kappa-statistic.png_

### Step 3: Calculate Fleiss' Kappa

While Cohen's Kappa is suitable for two annotators, for more than two annotators, we need to use Fleiss' Kappa.

Instead of implementing our own Fleiss' Kappa, we're going to use the [`statsmodels`](https://www.statsmodels.org/stable/index.html) library.

In [32]:
statsmodels.stats.inter_rater.aggregate_raters([annotations_szabi, annotations_antanas, annotations_kris], statsmodels.stats.inter_rater.nominal)

AttributeError: module 'statsmodels' has no attribute 'stats'

### Calculate average reported confidence

In [178]:
total_confidence_szabi = 0
count_confidence_szabi = 0

for s in annotations_szabi:
    if s['confidence'] is not None:
        total_confidence_szabi += s['confidence']
        count_confidence_szabi += 1
    
print(f"Average confidence Szabi: {total_confidence_szabi / count_confidence_szabi:.2f}")

total_confidence_antanas = 0
count_confidence_antanas = 0

for s in annotations_antanas:
    if s['confidence'] is not None:
        total_confidence_antanas += s['confidence']
        count_confidence_antanas += 1

print(f"Average confidence Antanas: {total_confidence_antanas / count_confidence_antanas:.2f}")

total_confidence_llm = 0
count_confidence_llm = 0

for s in annotations_baseline_llm:
    if s['confidence'] is not None:
        total_confidence_llm += s['confidence']
        count_confidence_llm += 1

print(f"Average confidence llm: {total_confidence_llm / count_confidence_llm:.2f}")

Average confidence Szabi: 2.36
Average confidence Antanas: 2.28
Average confidence llm: 2.91
